In [1]:
import pandas as pd
import os
import pyodbc
import xlsxwriter
import time
import scipy
from scipy.spatial import ConvexHull
import numpy as np
import sqlite3

In [2]:
os.chdir("C:/Users/ajpk/Desktop/Work Bundles")
os.getcwd()

'C:\\Users\\ajpk\\Desktop\\Work Bundles'

In [3]:
#Connect to MIB and get Pending FOs
start_time = time.time()

sql_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MSEDBSQLPRD01;DATABASE=MIB;Trusted_Connection=yes')

fd = open('non_complete_fos.sql', 'r')
query=fd.read()
                          
pend_fo_df=pd.read_sql_query(query, sql_conn)
sql_conn.close()
pend_fo_df.dropna()
end_time = time.time()
print(end_time -start_time)
pend_fo_df.head()

5.6830949783325195


,FO_ID,SP_ID,JOB_CODE,FO_CREATE_DATE,FO_COMPL_DATE,MTR_ID,MODEL_CD,PHASE,FORM_NBR,PREM_ID,...,PREM_LONGITUDE,MR_ROUTE_TYPE,MTR_READ_ROUTE,PHYSICAL_RTE,SUBAREA,OFFICE,DISPATCH_GROUP,SKILL_1,SKILL_2,JC_Work_Type
0,3303514943,3300431010,6080,2019-10-17,None,3279525766,AT-210,None,AT-210,3300431076,...,-122.367375,INTERVAL,WJT47SM,SJTF2,EJTS104,JT,Gas,GSR,None,Other
1,0657878216,0653407405,2142,2019-10-16,None,3873176698,I210+DSH,1,2S,0653407432,...,-121.855285,INTERVAL,SFB28SM,MFBH1,VFBS103,FB,FM,MMP,MST,65 Day E
2,1014340573,1016111826,2142,2019-10-01,None,4133617856,I210+DSH,1,2S,1016111811,...,-121.886627,INTERVAL,QJG20SM,NJGF3,DJGS109,JG,FM,MMP,MST,65 Day E
3,7575669986,7573943410,5236,2019-08-30,None,7573874488,R275,None,R275,7573943412,...,-119.726059,INTERVAL,CTK54SM,XTKF1,FTKS103,TK,FM,MMP,GCI,65 Day G
4,6853831567,6859141405,2180,2019-10-15,None,7067316248,FOCUSDSH,1,2S,6859141421,...,-119.243922,INTERVAL,LTJ24SM,YTJF5,FTJS103,TJ,FM,RMP,MMP,Chng Party Read/Verify


In [4]:
conn = sqlite3.connect(os.getcwd()+'/pending.sqlite')
pend_fo_df.to_sql('pending', conn, if_exists='replace', index=False)
conn.close()

In [4]:
#Connect to MIB and get avg time for FOs
start_time = time.time()

sql_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MSEDBSQLPRD01;DATABASE=MIB;Trusted_Connection=yes')

fd = open('weekly_fs_planning.sql', 'r')
query=fd.read()
                          
time_fo_df=pd.read_sql_query(query, sql_conn)
sql_conn.close()
end_time = time.time()
print(end_time -start_time)
time_fo_df.dropna()
time_fo_df.head()

42.437999963760376


,DIV,SKILL_1,JC_WORK_TYPE,NUMB_FO,FAS_TIME,AVG_TIME
0,Los Padres,GSR,Broken Lock,1,1.200000,1.200000
1,Diablo,MST,E Start Stop,126,95.616666,0.758862
2,Mission,GSR,Relight,3056,2489.583333,0.814654
3,North Bay,GSM,AC CGI,5,3.150000,0.630000
4,Stockton,MST,CTM/DA Inspection,2,1.666666,0.833333


In [10]:
sac_gasmm_df = pend_fo_df.loc[(pend_fo_df['DIV_NAME']=='Sacramento') & (pend_fo_df['JC_Work_Type']=='65 Day G')]
print(len(sac_gasmm_df))
sac_gasmm_df.head()

4151


,FO_ID,SP_ID,JOB_CODE,FO_CREATE_DATE,FO_COMPL_DATE,MTR_ID,MODEL_CD,PHASE,FORM_NBR,PREM_ID,...,MR_ROUTE_TYPE,MTR_READ_ROUTE,PHYSICAL_RTE,SUBAREA,OFFICE,DISPATCH_GROUP,SKILL_1,SKILL_2,JC_Work_Type,OFFICE_REMARK
13,5045114566,5047462605,5236,2019-06-14,None,4912009902,AC250,None,AC250,5047462656,...,INTERVAL,NPS78SM,JPSF1,SPSS110,PS,FM,MMP,GCI,65 Day G,AC250 SMOC Corrective Maintenance- MTU Failing...
20,4956549893,4950386405,5236,2019-03-22,None,5075089722,AC250,None,AC250,4950386466,...,INTERVAL,FPS39SM,KPSF1,SPSS105,PS,FM,MMP,GCI,65 Day G,AC250 SMOC Corrective Maintenance- Troubleshoo...
31,5103096124,5101388405,5236,2019-07-05,None,4871999112,R275,None,R275,5101388480,...,INTERVAL,SPS87SM,JPSF2,SPSS104,PS,FM,MMP,GCI,65 Day G,R275 SMOC Corrective Maintenance- MTU Failing-...
35,4867911164,4867310805,5236,2019-04-26,None,4846527985,R275,None,R275,4867310898,...,INTERVAL,GPS38SM,FPSF1,SPSS112,PS,FM,MMP,GCI,65 Day G,R275 SMOC Corrective Maintenance- MTU Failing-...
40,5095498024,5097817105,5236,2019-06-07,None,5014387312,AC250,None,AC250,5097817179,...,INTERVAL,JPS77SM,XPSF1,SPSS118,PS,FM,MMP,GCI,65 Day G,AC250 SMOC Corrective Maintenance- MTU Failing...
